In [59]:
#import global
import pprint
import os
import random
import numpy as np

### Preparing dataset

In [27]:
import csv

filePath = './blueplanet_dataset/scores.csv'
with open(filePath, 'r', encoding="utf8") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    scoresList = {row[0]: row[1] for row in csv_reader}

In [28]:
pprint.pprint(scoresList)

1uXxefX7xMkE-o.jpg': '3',
 '9505_q06a8212qjN4p0BKlq6-o.jpg': '3',
 '9506_q06a82bt4drcsinb4Ub-o.jpg': '3',
 '9507_q06a8y2h653252sOTtyU-o.jpg': '6',
 '9508_q06acnudqWlX9qH0WDn-o.jpg': '9',
 '9509_q06adw14w7CZnMsveI5-o.jpg': '6',
 '9510_q06adw9pgsgBEdoG8Zf-o.jpg': '6',
 '9511_q06adwak9AAPBV4Il7G-o.jpg': '6',
 '9512_q06adwhwhx7UQnXobXa-o.jpg': '9',
 '9513_q06adwpdp2ng12l23Cw-o.jpg': '4',
 '9514_q06adwxt8tiylc19Qo9-o.jpg': '1',
 '9515_q06afx11aqB7X7I73pHd-o.jpg': '6',
 '9516_q06afx2a38wRfnth2TGX-o.jpg': '8',
 '9517_q06afxpf6WAv6jiEQbf-o.jpg': '5',
 '9518_q06ajkbwvOhDjWD51t1-o.jpg': '9',
 '9519_q06ari1m33RqqlJKGEua-o.jpg': '8',
 '9520_q06ari9pp1ne59nFsJb-o.jpg': '9',
 '9521_q06arihyk7T9faDd3Sq-o.jpg': '6',
 '9522_q06ariztjtmGnTj1LRt-o.jpg': '4',
 '9523_q06atg2h6iCgvuW7WBTs-o.jpg': '6',
 '9524_q06atgampnm7Gp1rnlj-o.jpg': '6',
 '9525_q06atgbx02Hz5NsHl3Q-o.jpg': '6',
 '9526_q06atghz3NT55rY1w5v-o.jpg': '6',
 '9527_q06b1c2o9olo7PGNjHTC-o.jpg': '9',
 '9528_q06b1c2vbrdQr8RUouuK-o.jpg': '9',
 '9529_

In [26]:
# create categories - execute once
import shutil

categoryDir = './blueplanet_dataset/images/category/train/'
imgDir = './blueplanet_dataset/images/'
if not os.path.exists(categoryPath):
    os.makedirs(categoryPath)

for imgName, score in scoresList.items():
    catPath = categoryDir + score + '/'
    imgPath = imgDir + imgName
    print(imgPath, '-----', catPath)
    if not os.path.exists(catPath):
        os.makedirs(catPath)
    shutil.copy2(imgPath, catPath)

aset/images/9810_q06qm48om7euuU5jKbgJ-o.jpg ----- ./blueplanet_dataset/images/category/5/
./blueplanet_dataset/images/9811_q06qmrbx8lkMUKLovEC-o.jpg ----- ./blueplanet_dataset/images/category/9/
./blueplanet_dataset/images/9812_q06qne8onoiS4THlFsCH-o.jpg ----- ./blueplanet_dataset/images/category/8/
./blueplanet_dataset/images/9813_q06qo7753xKMmBSh3UaN-o.jpg ----- ./blueplanet_dataset/images/category/7/
./blueplanet_dataset/images/9814_q06qpw7qjpSNFSloTNfN-o.jpg ----- ./blueplanet_dataset/images/category/9/
./blueplanet_dataset/images/9815_q06qqfku1Ogpy839nwS-o.jpg ----- ./blueplanet_dataset/images/category/1/
./blueplanet_dataset/images/9816_q06qqkakb12ChlOuAyR-o.jpg ----- ./blueplanet_dataset/images/category/9/
./blueplanet_dataset/images/9817_q06qr5j1ytt5jF7g2Z3-o.jpg ----- ./blueplanet_dataset/images/category/8/
./blueplanet_dataset/images/9818_q06qt0xr37f7xxD31Q0-o.jpg ----- ./blueplanet_dataset/images/category/6/
./blueplanet_dataset/images/9819_q06qtixpxtz5xOj4sMg-o.jpg ----- ./

### Create Model

In [39]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
import os


weights_path = './blue-planet-pantip-analytics/python/image_pro/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

resnetModel = Sequential()
resnetModel.add(ResNet50(include_top=False, pooling='avg', weights=weights_path ))

In [40]:
num_classes = 10
resnetModel.add(Dense(num_classes, activation='softmax'))

# Say not to train first layer (ResNet) model. It is already trained
resnetModel.layers[0].trainable = False

In [43]:
resnetModel.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [44]:
from tensorflow.python.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import PIL

image_size = 224
trainDir = './blueplanet_dataset/images/category/train/'
valDir = './blueplanet_dataset/images/category/val/'
EPOCH = 5
BATCH_SIZE = 256
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = data_generator.flow_from_directory(
        trainDir,
        target_size=(image_size, image_size),
        batch_size=BATCH_SIZE,
        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
        valDir,
        target_size=(image_size, image_size),
        class_mode='categorical')

resnetModel.fit_generator(
        generator=train_generator,
        steps_per_epoch=train_generator.n//BATCH_SIZE,
        epochs=EPOCH,
        validation_data=validation_generator,
        validation_steps=validation_generator.n//BATCH_SIZE)

Found 5203 images belonging to 10 classes.
Found 547 images belonging to 10 classes.
Epoch 1/5
 1/20 [>.............................] - ETA: 41:50 - loss: 2.9338 - acc: 0.0898C:\Users\Vittunyuta\AppData\Local\Programs\Python\Python36\lib\site-packages\PIL\Image.py:989: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
20/20 [==============================] - 1709s 85s/step - loss: 2.3700 - acc: 0.1767 - val_loss: 2.3786 - val_acc: 0.2031
Epoch 2/5
20/20 [==============================] - 1602s 80s/step - loss: 2.0934 - acc: 0.2549 - val_loss: 2.1960 - val_acc: 0.2500
Epoch 3/5
20/20 [==============================] - 1610s 80s/step - loss: 1.9898 - acc: 0.2747 - val_loss: 2.1510 - val_acc: 0.2344
Epoch 4/5
20/20 [==============================] - 1581s 79s/step - loss: 1.9379 - acc: 0.2933 - val_loss: 2.1201 - val_acc: 0.2031
Epoch 5/5
20/20 [============================

#### Save model

In [49]:
SAVED_MODEL_DIR = './blue-planet-pantip-analytics/python/image_pro/saved_model/'
MODEL_NAME = 'scoresClassificationE5.hdf5'
log = resnetModel.save(SAVED_MODEL_DIR+MODEL_NAME)
if log == None:
    print('create file success')

None


#### Prediction

In [69]:
from keras.models import load_model

MODEL_NAME = 'scoresClassificationE5.hdf5'
path2savedModel = "./blue-planet-pantip-analytics/python/image_pro/saved_model/"
filePath = path2savedModel + MODEL_NAME

if isfile(filePath):
    model = load_model(filepath=filePath)
    model.summary()
else:
    raise Exception("--MODEL COULD NOT LOADED")

TypeError: load_model() missing 1 required positional argument: 'filepath'

In [56]:
# prepare data for prediction
from os.path import isfile, exists, isdir, join

path_dir = './blueplanet_dataset/images/category/train/'
imgPerCat = 2
imgsPath = []
catagory = [0,1,2,3,4,5,6,7,8,9]

for cat in catagory:
    path_ct = join(path_dir, str(cat)) + '/'

    image_list = os.listdir(path_ct)
    random.shuffle(image_list)
    
    for img in image_list[:imgPerCat]:
        path_img = join(path_ct,img)
        imgsPath.append({"path":path_img, "name":img, "actual":cat})

print(len(imgsPath))
pprint.pprint(imgsPath)

20
[{'actual': 0,
  'name': '9929_q07yyeji7dJMdKgKGk6-o.jpg',
  'path': './blueplanet_dataset/images/category/train/0/9929_q07yyeji7dJMdKgKGk6-o.jpg'},
 {'actual': 0,
  'name': '5514_pzzcudikdMIVH7tLEcG-o.jpg',
  'path': './blueplanet_dataset/images/category/train/0/5514_pzzcudikdMIVH7tLEcG-o.jpg'},
 {'actual': 1,
  'name': '1288_pztaqihl3SA38sxbz0V-o.jpg',
  'path': './blueplanet_dataset/images/category/train/1/1288_pztaqihl3SA38sxbz0V-o.jpg'},
 {'actual': 1,
  'name': '1447_pztd5jjbw4lmdCIyG1u-o.jpg',
  'path': './blueplanet_dataset/images/category/train/1/1447_pztd5jjbw4lmdCIyG1u-o.jpg'},
 {'actual': 2,
  'name': '9070_q04zi14s83e39cvhCHtv-o.jpg',
  'path': './blueplanet_dataset/images/category/train/2/9070_q04zi14s83e39cvhCHtv-o.jpg'},
 {'actual': 2,
  'name': '6021_q00rralwgwfWp7qbLSx-o.jpg',
  'path': './blueplanet_dataset/images/category/train/2/6021_q00rralwgwfWp7qbLSx-o.jpg'},
 {'actual': 3,
  'name': '9726_q06kp06lehIoW2kJyT9F-o.jpg',
  'path': './blueplanet_dataset/images/ca

In [62]:
from keras.preprocessing.image import load_img, img_to_array
match = 0
for idx, imgDetail in enumerate(imgsPath):
    img = load_img(path=imgDetail['path'], color_mode='rgb')
    img2predict = img.copy()
    img2predict = img_to_array(img2predict)
    img2predict = np.expand_dims(img2predict,0)
    img2predict /= 255
    pred = resnetModel.predict(img2predict)
    predout = np.round(pred, 3)
    predout = np.argmax(predout)
    # display(Image(img_path))
    if predout == imgDetail['actual']:
        match += 1
    print('{}.\n {}\n Prediction: {}\t {}\n Real Value: {}'.format(idx, imgDetail['name'], predout, pred, imgDetail['actual']))

print(match)
acc = match/len(imgsPath)
print(acc)

0.
 9929_q07yyeji7dJMdKgKGk6-o.jpg
 Prediction: 0	 [[9.1030133e-01 1.1236782e-03 4.6449163e-04 8.4129954e-03 6.7083077e-03
  1.2738982e-04 1.5797190e-02 2.9722873e-02 3.9506783e-03 2.3391226e-02]]
 Real Value: 0
1.
 5514_pzzcudikdMIVH7tLEcG-o.jpg
 Prediction: 0	 [[9.2628253e-01 1.1258449e-03 4.2064034e-04 6.6094021e-03 7.2538871e-03
  1.8601499e-04 1.6197847e-02 2.3272835e-02 3.1634900e-03 1.5487486e-02]]
 Real Value: 0
2.
 1288_pztaqihl3SA38sxbz0V-o.jpg
 Prediction: 0	 [[0.6418616  0.02005995 0.00587663 0.02673761 0.01449484 0.0071
  0.07147238 0.09794622 0.02425007 0.09020063]]
 Real Value: 1
3.
 1447_pztd5jjbw4lmdCIyG1u-o.jpg
 Prediction: 0	 [[0.86464834 0.00480094 0.00158636 0.01406342 0.00920524 0.00116031
  0.02301465 0.05780149 0.0060358  0.01768352]]
 Real Value: 1
4.
 9070_q04zi14s83e39cvhCHtv-o.jpg
 Prediction: 0	 [[9.2123365e-01 1.2008409e-03 3.8634299e-04 7.1540056e-03 4.7013275e-03
  9.8458659e-05 1.3191444e-02 2.9820256e-02 3.2096384e-03 1.9003902e-02]]
 Real Value: 2
5.
